In [224]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go 
import numpy as np
from scipy.stats import chi2_contingency

sns.set_style('whitegrid')

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


### Carga de dados

In [225]:
df_leads = pd.read_csv('./datasets/dataclass.csv')

In [226]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 37 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Prospect ID                                    9240 non-null   object 
 1   Lead Number                                    9240 non-null   int64  
 2   Lead Origin                                    9240 non-null   object 
 3   Lead Source                                    9204 non-null   object 
 4   Do Not Email                                   9240 non-null   object 
 5   Do Not Call                                    9240 non-null   object 
 6   Converted                                      9240 non-null   int64  
 7   TotalVisits                                    9103 non-null   float64
 8   Total Time Spent on Website                    9240 non-null   int64  
 9   Page Views Per Visit                           9103 

### Feature engineer

In [227]:
#remover as colunas propect ID e lead numbe
df_leads.drop(columns=['Prospect ID', 'Lead Number'], axis=1, inplace=True)

In [228]:
#mostrar e remover as colunas categoricas que possuem 1 valor possivel
for column in df_leads.select_dtypes(include=['object']).columns:
    if df_leads[column].nunique() == 1:
        print(f'coluna {column} possui somente um valor possivel: {df_leads[column].unique()}')
        df_leads.drop(columns=[column], axis=1, inplace=True)



coluna Magazine possui somente um valor possivel: ['No']
coluna Receive More Updates About Our Courses possui somente um valor possivel: ['No']
coluna Update me on Supply Chain Content possui somente um valor possivel: ['No']
coluna Get updates on DM Content possui somente um valor possivel: ['No']
coluna I agree to pay the amount through cheque possui somente um valor possivel: ['No']


In [229]:
#mostrar os valores possiveis para todas as colunas categoricas
for column in df_leads.select_dtypes(include=['object']).columns:
        print(f'coluna {column} valores possiveis {df_leads[column].unique()}')


coluna Lead Origin valores possiveis ['API' 'Landing Page Submission' 'Lead Add Form' 'Lead Import'
 'Quick Add Form']
coluna Lead Source valores possiveis ['Olark Chat' 'Organic Search' 'Direct Traffic' 'Google' 'Referral Sites'
 'Welingak Website' 'Reference' 'google' 'Facebook' nan 'blog'
 'Pay per Click Ads' 'bing' 'Social Media' 'WeLearn' 'Click2call'
 'Live Chat' 'welearnblog_Home' 'youtubechannel' 'testone' 'Press_Release'
 'NC_EDM']
coluna Do Not Email valores possiveis ['No' 'Yes']
coluna Do Not Call valores possiveis ['No' 'Yes']
coluna Last Activity valores possiveis ['Page Visited on Website' 'Email Opened' 'Unreachable'
 'Converted to Lead' 'Olark Chat Conversation' 'Email Bounced'
 'Email Link Clicked' 'Form Submitted on Website' 'Unsubscribed'
 'Had a Phone Conversation' 'View in browser link Clicked' nan
 'Approached upfront' 'SMS Sent' 'Visited Booth in Tradeshow'
 'Resubscribed to emails' 'Email Received' 'Email Marked Spam']
coluna Country valores possiveis [nan 'Ind

In [230]:
#mostrar o percentual de valores ausentes ou com valor igual a 'select para cada coluna categorica
for column in df_leads.select_dtypes(include=['object']).columns:
    contagem_nulas = (df_leads[column] == 'Select').sum() + df_leads[column].isnull().sum()
    print(f'{column}: {contagem_nulas / len(df_leads)*100:.2f}%')

Lead Origin: 0.00%
Lead Source: 0.39%
Do Not Email: 0.00%
Do Not Call: 0.00%
Last Activity: 1.11%
Country: 26.63%
Specialization: 36.58%
How did you hear about X Education: 78.46%
What is your current occupation: 29.11%
What matters most to you in choosing a course: 29.32%
Search: 0.00%
Newspaper Article: 0.00%
X Education Forums: 0.00%
Newspaper: 0.00%
Digital Advertisement: 0.00%
Through Recommendations: 0.00%
Tags: 36.29%
Lead Quality: 51.59%
Lead Profile: 74.19%
City: 39.71%
Asymmetrique Activity Index: 45.65%
Asymmetrique Profile Index: 45.65%
A free copy of Mastering The Interview: 0.00%
Last Notable Activity: 0.00%


In [231]:
# remover colunas nulas com maior de 25%
for column in df_leads.select_dtypes(include=['object']).columns:
    contagem_nulas = (df_leads[column] == 'Select').sum() + df_leads[column].isnull().sum()
    if(contagem_nulas / len(df_leads)*100) > 25:
        print(f'{column}: {contagem_nulas / len(df_leads)*100:.2f}%')
        df_leads.drop(columns=[column], axis=1, inplace=True)


Country: 26.63%
Specialization: 36.58%
How did you hear about X Education: 78.46%
What is your current occupation: 29.11%
What matters most to you in choosing a course: 29.32%
Tags: 36.29%
Lead Quality: 51.59%
Lead Profile: 74.19%
City: 39.71%
Asymmetrique Activity Index: 45.65%
Asymmetrique Profile Index: 45.65%


In [232]:
# Na coluna lead source substituir o valor google por Google
df_leads['Lead Source'] = df_leads['Lead Source'].apply(lambda x: 'Google' if x == 'google' else x)

In [233]:
# COnverter valores de colunas categoricas de yes no para 1 e 0
for column in df_leads.select_dtypes(include=[object]).columns:
    valores_unicos = df_leads[column].unique()
    if set(valores_unicos).issubset(set(['Yes', 'No'])):
        print(f'{column}')
        df_leads[column] = df_leads[column].apply(lambda x: 1 if x == 'Yes' else 0)

Do Not Email
Do Not Call
Search
Newspaper Article
X Education Forums
Newspaper
Digital Advertisement
Through Recommendations
A free copy of Mastering The Interview


In [234]:
df_leads.head(10)

,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Search,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Asymmetrique Activity Score,Asymmetrique Profile Score,A free copy of Mastering The Interview,Last Notable Activity
0,API,Olark Chat,0,0,0,0.0,0,0.0,Page Visited on Website,0,0,0,0,0,0,15.0,15.0,0,Modified
1,API,Organic Search,0,0,0,5.0,674,2.5,Email Opened,0,0,0,0,0,0,15.0,15.0,0,Email Opened
2,Landing Page Submission,Direct Traffic,0,0,1,2.0,1532,2.0,Email Opened,0,0,0,0,0,0,14.0,20.0,1,Email Opened
3,Landing Page Submission,Direct Traffic,0,0,0,1.0,305,1.0,Unreachable,0,0,0,0,0,0,13.0,17.0,0,Modified
4,Landing Page Submission,Google,0,0,1,2.0,1428,1.0,Converted to Lead,0,0,0,0,0,0,15.0,18.0,0,Modified
5,API,Olark Chat,0,0,0,0.0,0,0.0,Olark Chat Conversation,0,0,0,0,0,0,17.0,15.0,0,Modified
6,Landing Page Submission,Google,0,0,1,2.0,1640,2.0,Email Opened,0,0,0,0,0,0,14.0,20.0,0,Modified
7,API,Olark Chat,0,0,0,0.0,0,0.0,Olark Chat Conversation,0,0,0,0,0,0,15.0,15.0,0,Modified
8,Landing Page Submission,Direct Traffic,0,0,0,2.0,71,2.0,Email Opened,0,0,0,0,0,0,14.0,14.0,1,Email Opened
9,API,Google,0,0,0,4.0,58,4.0,Email Opened,0,0,0,0,0,0,13.0,16.0,0,Email Opened


In [235]:
#remover linhas com valores ausentes 
colunas_categoricas = df_leads.select_dtypes(include=['object']).columns
df_leads.dropna(subset=colunas_categoricas, inplace=True)

In [236]:
df_leads.describe()

,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Search,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Asymmetrique Activity Score,Asymmetrique Profile Score,A free copy of Mastering The Interview
count,9103.000000,9103.000000,9103.000000,9074.000000,9103.000000,9074.000000,9103.000000,9103.000000,9103.000000,9103.000000,9103.000000,9103.000000,4944.000000,4944.000000,9103.000000
mean,0.079205,0.000220,0.379216,3.456028,483.773921,2.370151,0.001538,0.000220,0.000110,0.000110,0.000439,0.000769,14.313511,16.340210,0.317258
std,0.270073,0.014822,0.485219,4.858802,545.519186,2.160871,0.039189,0.014822,0.010481,0.010481,0.020959,0.027721,1.394627,1.807428,0.465434
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,11.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,12.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,15.000000,0.000000
50%,0.000000,0.000000,0.000000,3.000000,247.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,16.000000,0.000000
75%,0.000000,0.000000,1.000000,5.000000,924.000000,3.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000,18.000000,1.000000
max,1.000000,1.000000,1.000000,251.000000,2272.000000,55.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,18.000000,20.000000,1.000000


In [237]:
#mostrar e remover as colunas numericas que possuem 1 valor possivel
for column in df_leads.select_dtypes(include=['number']).columns:
    if df_leads[column].nunique() == 1:
        print(f'coluna {column} possui somente um valor possivel: {df_leads[column].unique()}')
        df_leads.drop(columns=[column], axis=1, inplace=True)



In [238]:
#mostrar o percentual de valores ausentes ou com valor igual a 'select para cada coluna categorica
for column in df_leads.select_dtypes(include=['number']).columns:
    contagem_nulas = (df_leads[column] == 'Select').sum() + df_leads[column].isnull().sum()
    print(f'{column}: {contagem_nulas / len(df_leads)*100:.2f}%')

Do Not Email: 0.00%
Do Not Call: 0.00%
Converted: 0.00%
TotalVisits: 0.32%
Total Time Spent on Website: 0.00%
Page Views Per Visit: 0.32%
Search: 0.00%
Newspaper Article: 0.00%
X Education Forums: 0.00%
Newspaper: 0.00%
Digital Advertisement: 0.00%
Through Recommendations: 0.00%
Asymmetrique Activity Score: 45.69%
Asymmetrique Profile Score: 45.69%
A free copy of Mastering The Interview: 0.00%


In [239]:
# remover colunas nulas com maior de 25%
for column in df_leads.select_dtypes(include=['number']).columns:
    contagem_nulas = df_leads[column].isnull().sum()
    if(contagem_nulas / len(df_leads)*100) > 25:
        print(f'{column}: {contagem_nulas / len(df_leads)*100:.2f}%')
        df_leads.drop(columns=[column], axis=1, inplace=True)


Asymmetrique Activity Score: 45.69%
Asymmetrique Profile Score: 45.69%


In [240]:
#remover linhas com valores ausentes 
colunas_numericas = df_leads.select_dtypes(include=['number']).columns
df_leads.dropna(subset=colunas_numericas, inplace=True)

### EDA

In [241]:
#hit ratio 
# Conceito 1 = razao entre leads convertido em vendas sobre quantidade total de leads
# 100 leads foram criadas e 30 foram convertidos = 30%
# Conceito 2 = razao entre leads convertidos em vendas sobre quantidade de leads encerrados
# 100 leads criados, 20 nao convert e 20 foram convertidos


In [242]:
# Distribuição target em percentual
fig = px.bar(df_leads['Converted'].value_counts()/len(df_leads)*100, title='Hit Ratio - Fator de conversão')
fig.show()

In [243]:
#plot de correlação das variaveis numericas com plotly 
corr_matrix = df_leads.select_dtypes(include=['number']).corr()


In [244]:
fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x = corr_matrix.columns,
        y = corr_matrix.index,
        z = np.array(corr_matrix),
        text = corr_matrix.values,
        texttemplate='%{text:.2f}',
        colorscale=px.colors.diverging.RdBu,
        zmin=-1,
        zmax=1
    )
)

fig.show()

In [245]:
#boxplot coverted x totalvisits
fig = px.box(df_leads, x='Converted', y='TotalVisits', color='Converted')
fig.show()

In [246]:
#boxplot coverted x totalvisits
fig = px.box(df_leads, x='Converted', y='Total Time Spent on Website', color='Converted')
fig.show()

In [247]:
#boxplot coverted x totalvisits
fig = px.box(df_leads, x='Converted', y='Page Views Per Visit', color='Converted')
fig.show()

In [248]:
##criar uma tabela de contingencia
contigency_table_lead_source = pd.crosstab(df_leads['Converted'], df_leads['Lead Source'])

In [249]:
contigency_table_lead_source

Lead Source,Click2call,Direct Traffic,Facebook,Google,Live Chat,NC_EDM,Olark Chat,Organic Search,Pay per Click Ads,Press_Release,Reference,Referral Sites,Social Media,WeLearn,Welingak Website,bing,blog,testone,welearnblog_Home,youtubechannel
Converted,,,,,,,,,,,,,,,,,,,,
0,1,1725,22,1726,0,0,1305,718,1,2,33,94,1,0,2,5,1,1,1,1
1,3,818,9,1147,2,1,448,436,0,0,410,31,1,1,127,1,0,0,0,0


In [250]:
# Executar o teste de independencia quiquadrado
chi2, p, dof, expected = chi2_contingency(contigency_table_lead_source)

In [251]:
print(f'quiquadrado: {chi2}')
print(f'Valor p: {p}')
print(f'Grau de liberdade: {dof}')
print(f'Existe uma relação significativa entre converted e lead source? {p < 0.05}')

quiquadrado: 942.1372507753774
Valor p: 1.1748671316223743e-187
Grau de liberdade: 19
Existe uma relação significativa entre converted e lead source? True


In [252]:
##criar uma tabela de contingencia
contigency_table_lead_origin = pd.crosstab(df_leads['Converted'], df_leads['Lead Origin'])

In [253]:
contigency_table_lead_origin

Lead Origin,API,Landing Page Submission,Lead Add Form,Lead Import
Converted,,,,
0,2463,3118,37,21
1,1115,1767,544,9


In [254]:
# Executar o teste de independencia quiquadrado
chi2, p, dof, expected = chi2_contingency(contigency_table_lead_origin)

In [255]:
print(f'quiquadrado: {chi2}')
print(f'Valor p: {p}')
print(f'Grau de liberdade: {dof}')
print(f'Existe uma relação significativa entre converted e lead origin? {p < 0.05}')

quiquadrado: 843.1212236836468
Valor p: 1.9228780932726904e-182
Grau de liberdade: 3
Existe uma relação significativa entre converted e lead origin? True


### Preparação para os dados 

In [256]:
X = df_leads.drop(columns=['Converted'])
y = df_leads['Converted']

In [257]:
#criar transformer
numeric_features = X.select_dtypes(include=['number']).columns

categorical_features = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

In [258]:
#dividr os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=51)

#aplicar colum transformer
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

### Treinamento do modelo



In [259]:
#criar o modelo de bagging Classifier
bagging_model = BaggingClassifier(
    estimator=LogisticRegression(), 
    n_estimators=10,
    random_state=51,
    max_samples=0.3,
    max_features=0.3
)

In [260]:
#treinar
bagging_model.fit(X_train, y_train)

,estimator,LogisticRegression()
,n_estimators,10
,max_samples,0.3
,max_features,0.3
,bootstrap,True
,bootstrap_features,False
,oob_score,False
,warm_start,False
,n_jobs,None
,random_state,51
,verbose,0


### Avaliação do modelo

In [261]:
#fazer predições no conjunto de teste
y_pred = bagging_model.predict(X_test)

In [262]:
#avaliar modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [263]:
print(f'Acurária: {accuracy}')
print(f'Precisão: {precision}')
print(f'Recall: {recall}')
print(f'f1: {f1}')

Acurária: 0.7509641873278237
Precisão: 0.7422222222222222
Recall: 0.49850746268656715
f1: 0.5964285714285714


In [264]:
#mostrar uma matriz de confusao
conf_matrix = confusion_matrix(y_test, y_pred)

fig = px.imshow(conf_matrix,
                labels=dict(x='Predição', y='Real', coor='Contagem'),
                x=['Not Converted', 'Converted'],
                y=['Not Converted', 'Converted'])

fig.update_traces(text=conf_matrix, texttemplate="%{z}")
fig.update_layout(coloraxis_showscale=False)
fig.show()

In [265]:
#Calcular a importancia das variaveis 
importances = np.mean([np.abs(estimator.coef_[0]) for estimator in bagging_model.estimators_], axis=0)
importances

array([0.7216204 , 0.64001581, 0.48853284, 0.79929892, 0.2419471 ,
       0.41312859, 0.6226342 , 0.38897817, 0.7159801 , 0.54371071,
       0.52518225, 0.30667573, 0.6687867 , 0.46177705, 0.72763512,
       0.4430405 , 0.15686804, 0.46183659, 0.49061144, 0.48726468])

In [266]:
#obter os nomes das features 
features_names = (numeric_features.tolist() +
                  preprocessor.named_transformers_['cat']
                  .get_feature_names_out(categorical_features).tolist())

In [267]:
features_names

['Do Not Email',
 'Do Not Call',
 'TotalVisits',
 'Total Time Spent on Website',
 'Page Views Per Visit',
 'Search',
 'Newspaper Article',
 'X Education Forums',
 'Newspaper',
 'Digital Advertisement',
 'Through Recommendations',
 'A free copy of Mastering The Interview',
 'Lead Origin_API',
 'Lead Origin_Landing Page Submission',
 'Lead Origin_Lead Add Form',
 'Lead Origin_Lead Import',
 'Lead Source_Click2call',
 'Lead Source_Direct Traffic',
 'Lead Source_Facebook',
 'Lead Source_Google',
 'Lead Source_Live Chat',
 'Lead Source_NC_EDM',
 'Lead Source_Olark Chat',
 'Lead Source_Organic Search',
 'Lead Source_Pay per Click Ads',
 'Lead Source_Reference',
 'Lead Source_Referral Sites',
 'Lead Source_Social Media',
 'Lead Source_WeLearn',
 'Lead Source_Welingak Website',
 'Lead Source_bing',
 'Lead Source_blog',
 'Lead Source_testone',
 'Lead Source_welearnblog_Home',
 'Lead Source_youtubechannel',
 'Last Activity_Approached upfront',
 'Last Activity_Converted to Lead',
 'Last Activity_

In [268]:
#criar um df nome das features e importancias
df_feature_importances = pd.DataFrame({'Feature': features_names, 'Importance': importances})

ValueError: All arrays must be of the same length

In [ ]:
#ordenar odataframe pela importancia
df_feature_importances = df_feature_importances.sort_values(by='Importance', ascending=True)

In [ ]:
#plotar a importancia das features
fig = px.bar(df_feature_importances,
             x='Importance',
             y='Feature',
             orientation='h')

fig.update_layout(height=1280, width=1000, yaxis={'categoryorder': 'total ascending'})

### Some more things

In [ ]:
bagging_model.estimators_samples_

[array([6647, 2395, 3964, ..., 5215, 2295,  480]),
 array([ 914, 6214, 4940, ...,  111, 6462, 2927]),
 array([6067, 5889, 4247, ..., 5093,  220, 6752]),
 array([ 405, 6461, 1538, ..., 2355, 1434, 5213]),
 array([3226, 4034, 4872, ...,  587, 5567, 4657])]

In [ ]:
bagging_model.estimators_samples_[0].shape

(2177,)

In [ ]:
bagging_model.estimators_features_

[array([ 5, 12,  7,  2, 46, 40, 45,  9,  8, 56,  4, 54, 32, 63, 53, 34, 51,
        33, 59, 50]),
 array([13, 19, 63, 34, 43, 30, 67, 29, 55, 21,  7, 52, 47, 59, 14, 56, 66,
        57, 39, 27]),
 array([10,  3,  5, 52, 20, 41, 63, 17, 37,  2, 44, 45,  9, 32, 57, 48, 40,
        24, 25, 16]),
 array([11, 24, 51, 48,  6, 64, 34, 60, 45, 25, 63, 67, 17, 27, 14, 56, 57,
         5, 41, 43]),
 array([13, 19, 59, 62, 50,  4,  7, 23, 47, 64, 21,  9, 12, 29, 35, 52,  2,
        45, 28, 51])]

In [ ]:
#predizer probabilidade de conversão
y_pred_prob = bagging_model.predict_proba(X_test)

In [ ]:
y_pred_prob

array([[0.45416619, 0.54583381],
       [0.72937115, 0.27062885],
       [0.72937115, 0.27062885],
       ...,
       [0.7631031 , 0.2368969 ],
       [0.75700224, 0.24299776],
       [0.3712033 , 0.6287967 ]])

## Cenarios de CRM - Utilidade da probabilidade

CRM
- Leads concluidos = Resultado positivo ou negativo
- Leads em aberto = nâo tenho Resultado

Treine um modelo no que está concluido para que ele generalize bem no que está em aberto

Lead em aberto 
- Probabilidade de Converter
Quando muito alto, podemos olhar com mais foco pra realmente converter, quando muito baixo, podemos meio que descartar
- Importância das Features


In [270]:
df_leads.to_csv('./datasets/leads_cleaned.csv', index=False)

In [271]:
import joblib

joblib.dump(preprocessor, './preprocessor_leads.pkl')

['./preprocessor_leads.pkl']